# Inheco ODTC (On Deck Thermal Cycler)

The Inheco ODTC is an on-deck thermal cycler designed for automated PCR workflows. It features:

- Precise temperature control for PCR cycling
- Heated lid to prevent condensation
- Motorized door for automated plate handling
- SiLA 2 communication interface

**Specifications:**
- Temperature range: 4°C to 99°C
- Heating/cooling rate: up to 4.4°C/s
- 96-well plate format

See the [Inheco ODTC product page](https://www.inheco.com/odtc.html) for more information.

---

## Setup

The ODTC communicates over Ethernet using the SiLA 2 protocol. You'll need:
1. The IP address of the ODTC
2. Network connectivity between your computer and the ODTC

In [ ]:
from pylabrobot.thermocycling.inheco import ODTC

odtc = ODTC(
  name="odtc",
  ip="169.254.151.99",  # Replace with your ODTC's IP address
)
await odtc.setup()

---

## Door Control

Open and close the door for plate access:

In [ ]:
await odtc.open_lid()

In [ ]:
await odtc.close_lid()

---

## Temperature Control

### Reading Sensor Data

Get current temperatures from all sensors:

In [ ]:
sensor_data = await odtc.get_sensor_data()
print(sensor_data)
# Example output:
# {'Mount': 25.0, 'Mount_Monitor': 25.1, 'Lid': 30.0, 'Lid_Monitor': 30.1,
#  'Ambient': 22.0, 'PCB': 28.0, 'Heatsink': 26.0, 'Heatsink_TEC': 25.5}

### Setting Block Temperature

Set a constant block temperature. Note that the ODTC uses a "pre-method" approach which takes several minutes to stabilize:

In [ ]:
await odtc.set_block_temperature([37.0])  # Set to 37°C

In [ ]:
# Check current block temperature
temp = await odtc.get_block_current_temperature()
print(f"Block temperature: {temp[0]}°C")

### Deactivating Temperature Control

In [ ]:
await odtc.deactivate_block()

---

## Running PCR Protocols

The ODTC can run complex PCR protocols defined using `Protocol`, `Stage`, and `Step` objects.

### Defining a Protocol

A protocol consists of stages, each containing steps with temperature and hold time. Stages can repeat multiple times for cycling.

In [ ]:
from pylabrobot.thermocycling.standard import Protocol, Stage, Step

# Example: Standard 3-step PCR protocol
pcr_protocol = Protocol(
  stages=[
    # Initial denaturation
    Stage(
      steps=[Step(temperature=[95.0], hold_seconds=300)],  # 95°C for 5 min
      repeats=1
    ),
    # PCR cycling (30 cycles)
    Stage(
      steps=[
        Step(temperature=[95.0], hold_seconds=30),   # Denature: 95°C for 30s
        Step(temperature=[55.0], hold_seconds=30),   # Anneal: 55°C for 30s
        Step(temperature=[72.0], hold_seconds=60),   # Extend: 72°C for 60s
      ],
      repeats=30
    ),
    # Final extension
    Stage(
      steps=[Step(temperature=[72.0], hold_seconds=600)],  # 72°C for 10 min
      repeats=1
    ),
    # Hold
    Stage(
      steps=[Step(temperature=[4.0], hold_seconds=0)],  # 4°C hold
      repeats=1
    ),
  ]
)

### Running the Protocol

In [ ]:
await odtc.run_protocol(
  protocol=pcr_protocol,
  block_max_volume=20.0,           # Maximum sample volume in µL
  start_block_temperature=25.0,    # Starting block temperature
  start_lid_temperature=105.0,     # Lid temperature (typically 105°C to prevent condensation)
)

### Custom Ramp Rates

You can specify custom temperature ramp rates for each step:

In [ ]:
# Protocol with custom ramp rates
custom_protocol = Protocol(
  stages=[
    Stage(
      steps=[
        Step(temperature=[95.0], hold_seconds=60, rate=4.4),  # Fast ramp (4.4°C/s)
        Step(temperature=[60.0], hold_seconds=30, rate=2.0),  # Slower ramp (2.0°C/s)
      ],
      repeats=1
    ),
  ]
)

In [ ]:
await odtc.run_protocol(
  protocol=custom_protocol,
  block_max_volume=25.0,
  start_block_temperature=25.0,
  start_lid_temperature=105.0,
)

---

## Closing the Connection

In [ ]:
await odtc.stop()